Differential Drive: robot with two wheels driving it. 

Base link: main coordinate system: x-pointing forward, y-pointing left, z-pointing up.

![Robot State Publisher](./images/Robot_state_publisher.png)

#### Running GUI
xhost +local:  
In terminal, and rebuilding container.

Don't install Python/Anaconda

cd my_bot && mkdir src
git clone https://github.com/SunzidHassan/my_bot.git
cd my_bot
- pip install catkin_pkg
colcon build --symlink-install


#### Building Xacro robot description
Everytime we add a new file:  
colcon build --symlink-install

#### State publisher
Then we need to source the updated install in state publisher, gazebo, etc.:  
source install/setup.bash

State publisher: 
ros2 launch my_bot rsp.launch.py use_sim_time:=true

#### Install Gazebo
sudo apt update && sudo apt upgrade -y
sudo apt install -y ros-humble-gazebo-ros-pkgs


#### Gazebo simulation
rosdep update

Gazebo launch:  
ros2 launch gazebo_ros gazebo.launch.py  

Launching with launcher file:  
ros2 launch my_bot launch_sim.launch.py

If the normal or 3d camera is not working, try
- colcon build and source setup.bash.
- starting from a fresh gazebo env,
- install python

Launching with launcher in custom simulation environment:  
ros2 launch my_bot launch_sim.launch.py world:=./src/my_bot/worlds/football.world 

#### Rviz load view
rviz2 -d src/my_bot/config/view_bot.rviz

#### Teleop
ros2 run teleop_twist_keyboard teleop_twist_keyboard

![Robot Control](.images/robot_control_1.png)

## Camera
ros2 run rqt_image_view rqt_image_view

ros2 run image_transport republish compressed raw --ros-args -r in/compressed:=/camera/image_raw/compressed -r out:=/camera/image_raw/uncompressed

## Cuda-Pytorch
- Install nvidia container toolkit
- Add "--gpus","all" in "runArgs"
  "runArgs": [
    "--privileged",
    "--network=host",
    "--gpus","all"
  ],
- Install correct cuda for latest PyTorch, install default Pytorch

# Docker image tries

## my_bot
Disconnect VPN

{
  "name": "humble desktop-full",
  "dockerFile": "Dockerfile",
  "runArgs": [
    "--privileged",
    "--network=host",
    "--gpus","all"
  ],
  "workspaceMount": "source=${localWorkspaceFolder},target=/${localWorkspaceFolderBasename},type=bind",
  "workspaceFolder": "/${localWorkspaceFolderBasename}",
  "mounts": [
    "source=${localEnv:HOME}${localEnv:USERPROFILE}/.bash_history,target=/home/vscode/.bash_history,type=bind"
],
"features": {
	"ghcr.io/devcontainers/features/desktop-lite:1": {},
  "ghcr.io/devcontainers/features/nvidia-cuda:1": {},
  "ghcr.io/raucha/devcontainer-features/pytorch:1": {}
}
}


In [ ]:
FROM osrf/ros:humble-desktop-full

# Add vscode user with same UID and GID as your host system
# (copied from https://code.visualstudio.com/remote/advancedcontainers/add-nonroot-user#_creating-a-nonroot-user)
ARG USERNAME=vscode
ARG USER_UID=1000
ARG USER_GID=$USER_UID
RUN groupadd --gid $USER_GID $USERNAME \
    && useradd -s /bin/bash --uid $USER_UID --gid $USER_GID -m $USERNAME \
    && apt-get update \
    && apt-get install -y sudo \
    && echo $USERNAME ALL=\(root\) NOPASSWD:ALL > /etc/sudoers.d/$USERNAME \
    && chmod 0440 /etc/sudoers.d/$USERNAME
# Switch from root to user
USER $USERNAME

# Add user to video group to allow access to webcam
RUN sudo usermod --append --groups video $USERNAME

# Update all packages
RUN sudo apt update && sudo apt upgrade -y

# Install Git
RUN sudo apt install -y git

# Rosdep update
RUN rosdep update

# Source the ROS setup file
RUN echo "source /opt/ros/${ROS_DISTRO}/setup.bash" >> ~/.bashrc

# Install wget
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y wget

# Add workspace directory
RUN cd my_bot && mkdir src
RUN git clone https://github.com/SunzidHassan/my_bot.git
RUN cd my_bot
RUN pip install catkin_pkg
RUN colcon build --symlink-install

# Install gazebo
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-gazebo-ros-pkgs


## Turtlebot4

In [ ]:
#  Command to build it
# docker built -t <image name > .
FROM osrf/ros:humble-desktop-full

# Add vscode user with same UID and GID as your host system
# (copied from https://code.visualstudio.com/remote/advancedcontainers/add-nonroot-user#_creating-a-nonroot-user)
ARG USERNAME=vscode
ARG USER_UID=1000
ARG USER_GID=$USER_UID
RUN groupadd --gid $USER_GID $USERNAME \
    && useradd -s /bin/bash --uid $USER_UID --gid $USER_GID -m $USERNAME \
    && apt-get update \
    && apt-get install -y sudo \
    && echo $USERNAME ALL=\(root\) NOPASSWD:ALL > /etc/sudoers.d/$USERNAME \
    && chmod 0440 /etc/sudoers.d/$USERNAME
# Switch from root to user
USER $USERNAME

# Add user to video group to allow access to webcam
RUN sudo usermod --append --groups video $USERNAME

# Update all packages
RUN sudo apt update && sudo apt upgrade -y

# Install Git
RUN sudo apt install -y git && apt-get install -y python3-pip

# Rosdep update (disconnect vpn)
RUN rosdep update

RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-dev-tools

# Install wget
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y wget

# Install Ingition
RUN sudo sh -c 'echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-stable `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-stable.list'
RUN wget http://packages.osrfoundation.org/gazebo.key -O - | sudo apt-key add -
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y ignition-fortress

# Install simulator
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-turtlebot4-simulator

# Add workspace directory
RUN mkdir -p ~/turtlebot4_ws/src && cd ~/turtlebot4_ws/src/

RUN git clone https://github.com/turtlebot/turtlebot4_simulator.git -b humble

RUN cd ..

RUN sudo rosdep init
RUN rosdep update

RUN rosdep install -y --from-path src -yi

# Install catkin_pkg before colcon build
RUN sudo apt update && sudo apt upgrade -y
RUN pip install catkin_pkg

# Source the ROS setup file
RUN echo "source /opt/ros/humble/setup.bash" >> ~/.bashrc

RUN colcon build --symlink-install


RUN echo "ALL Done"

## Turtlebot3

In [ ]:
#  Command to build it
# docker built -t <image name > .
FROM osrf/ros:humble-desktop-full

# Add vscode user with same UID and GID as your host system
# (copied from https://code.visualstudio.com/remote/advancedcontainers/add-nonroot-user#_creating-a-nonroot-user)
ARG USERNAME=vscode
ARG USER_UID=1000
ARG USER_GID=$USER_UID
RUN groupadd --gid $USER_GID $USERNAME \
    && useradd -s /bin/bash --uid $USER_UID --gid $USER_GID -m $USERNAME \
    && apt-get update \
    && apt-get install -y sudo \
    && echo $USERNAME ALL=\(root\) NOPASSWD:ALL > /etc/sudoers.d/$USERNAME \
    && chmod 0440 /etc/sudoers.d/$USERNAME
# Switch from root to user
USER $USERNAME

# Add user to video group to allow access to webcam
RUN sudo usermod --append --groups video $USERNAME

# Update all packages
RUN sudo apt update && sudo apt upgrade -y

# Install Git
RUN sudo apt install -y git

RUN sudo apt install -y python3

# Rosdep update
RUN rosdep update

RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-dev-tools

# Install wget
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y wget

############################################################
## Turtlebot3 Simulation
# Install Ingition
RUN sudo sh -c 'echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-stable `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-stable.list'
RUN wget http://packages.osrfoundation.org/gazebo.key -O - | sudo apt-key add -
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y ignition-fortress

# Install simulator
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-turtlebot4-simulator

# Add workspace directory
RUN mkdir -p ~/turtlebot4_ws/src && cd ~/turtlebot4_ws/src/

RUN git clone https://github.com/turtlebot/turtlebot4_simulator.git -b humble

RUN cd ..

RUN sudo rosdep init
RUN rosdep update

RUN rosdep install -y --from-path src -yi

# Install catkin_pkg before colcon build
RUN sudo apt update && sudo apt upgrade -y
RUN pip install catkin_pkg

# Source the ROS setup file
RUN echo "source /opt/ros/humble/setup.bash" >> ~/.bashrc

RUN colcon build --symlink-install


############################################################
## Turtlebot3 Simulation

# Gazebo install
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-gazebo-*

# Cartographer
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-cartographer
RUN sudo apt install -y ros-humble-cartographer-ros

# Navigation 2
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-navigation2
RUN sudo apt install -y ros-humble-nav2-bringup

# Turtlebot3 packages
RUN sudo apt update && sudo apt upgrade -y
RUN source ~/.bashrc
RUN sudo apt install -y ros-humble-dynamixel-sdk
RUN sudo apt install -y ros-humble-turtlebot3-msgs
RUN sudo apt install -y ros-humble-turtlebot3

# Environment configuration
RUN sudo apt update && sudo apt upgrade -y
RUN echo 'export ROS_DOMAIN_ID=30 #TURTLEBOT3' >> ~/.bashrc
RUN source ~/.bashrc

# Turtlebot3 simulation
RUN cd ~/turtlebot3/src
RUN git clone -b humble-devel https://github.com/ROBOTIS-GIT/turtlebot3_simulations.git
RUN cd ~/turtlebot3_ws && colcon build --symlink-install


RUN echo "ALL Done"

Build the dockerfile and get Docker image ID: docker build -t r2_humble_turtlebot34 .

make the bash file and Dockerfile executable

./run_docker_gpu.bash

In [ ]:
xhost local:root

XAUTH=/tmp/.docker.xauth

docker run -it \
    --name=turtlebot4_simulator_container \
    --env="DISPLAY=$DISPLAY" \
    --env="QT_X11_NO_MITSHM=1" \
    --volume="/tmp/.X11-unix:/tmp/.X11-unix:rw" \
    --env="XAUTHORITY=$XAUTH" \
    --volume="$XAUTH:$XAUTH" \
    --net=host \
    --privileged \
    --runtime=nvidia \
    r2_humble_turtlebot34 \
    bash

echo "Done."